In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, Add
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Concatenate, Dropout, Reshape, Multiply

# Define the FCA block
# FCA is used to emphasize important channels and suppress less important ones.
def FCA_block(inputs, reduction_ratio=16):
    # Get the number of channels in the input tensor
    channel_axis = -1
    channels = inputs.shape[channel_axis]

    # Apply Global Average Pooling to the input tensor to get a channel-wise summary
    x = GlobalAveragePooling2D()(inputs)

    # Reduce the number of channels by a factor of `reduction_ratio` using a fully connected layer
    x = Dense(channels // reduction_ratio, activation='relu')(x)

    # Restore the number of channels to the original using another fully connected layer
    x = Dense(channels, activation='sigmoid')(x)

    # Reshape the output to match the input tensor's shape but with single spatial dimensions
    x = Reshape((1, 1, channels))(x)

    # Multiply the input tensor with the output of the fully connected layers
    # This step modulates the importance of each channel
    outputs = Multiply()([inputs, x])
    return outputs

# Define a residual block with FCA
# This block uses residual connections to improve gradient flow and an FCA block for channel attention.
def residual_block(inputs, filters, kernel_size=3, strides=1, use_fca=True):
    # Apply the first convolutional layer
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(inputs)

    # Apply batch normalization to stabilize the training process
    x = BatchNormalization()(x)

    # Apply the LeakyReLU activation function for non-linearity
    x = LeakyReLU(alpha=0.2)(x)

    # Optionally apply the FCA block
    if use_fca:
        x = FCA_block(x)

    # Apply the second convolutional layer
    x = Conv2D(filters, kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # Create a shortcut connection that adds the input directly to the output if the dimensions match
    if strides != 1 or inputs.shape[-1] != filters:
        # If dimensions do not match, apply a convolution to the input to match the output dimensions
        shortcut = Conv2D(filters, kernel_size=1, strides=strides, padding='same')(inputs)
    else:
        # Otherwise, the shortcut is just the input
        shortcut = inputs

    # Add the shortcut to the output of the second convolutional layer
    x = Add()([x, shortcut])
    return x

# Define the full model
def build_model(input_shape=(128, 128, 3), num_classes=5):
    inputs = Input(shape=input_shape)  # Define the input layer with the given shape

    # Initial convolutional layer to extract basic features
    x = Conv2D(64, (7, 7), strides=2, padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)  # Apply max pooling to reduce spatial dimensions
    x = BatchNormalization()(x)  # Normalize the activations

    # Add residual blocks with increasing complexity and spatial downsampling
    x = residual_block(x, 64)  # First block, keeps the spatial dimensions
    x = residual_block(x, 64)  # Second block, keeps the spatial dimensions
    x = residual_block(x, 128, strides=2)  # Third block, reduces the spatial dimensions by half
    x = residual_block(x, 128)  # Fourth block, keeps the spatial dimensions
    x = residual_block(x, 256, strides=2)  # Fifth block, reduces the spatial dimensions by half
    x = residual_block(x, 256)  # Sixth block, keeps the spatial dimensions
    x = residual_block(x, 512, strides=2)  # Seventh block, reduces the spatial dimensions by half
    x = residual_block(x, 512)  # Eighth block, keeps the spatial dimensions

    # Apply global average pooling to reduce each feature map to a single value
    x = GlobalAveragePooling2D()(x)

    # Fully connected layers for final classification
    x = Dense(512, activation='relu')(x)  # Dense layer with 512 units and ReLU activation
    x = Dropout(0.5)(x)  # Dropout for regularization
    x = Dense(64, activation='relu')(x)  # Dense layer with 64 units and ReLU activation
    x = Dropout(0.5)(x)  # Dropout for regularization
    x = Dense(16, activation='relu')(x)  # Dense layer with 16 units and ReLU activation
    x = Dropout(0.5)(x)  # Dropout for regularization

    # Final output layer with softmax activation for classification
    outputs = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs, outputs)
    return model

# Build and compile the model
input_shape = (128, 128, 3)  # Define the input shape of the images
num_classes = 5  # Define the number of output classes
model = build_model(input_shape, num_classes)

# Compile the model with Adam optimizer and sparse categorical crossentropy loss
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to see the architecture
model.summary()


2024-06-12 18:26:46.248271: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 18:26:46.450435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 18:26:46.450505: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 18:26:46.483433: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-12 18:26:46.539170: I tensorflow/core/platform/cpu_feature_guar

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 64, 64, 64)           9472      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 32, 32, 64)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 64)           256       ['max_pooling2d[0][0]']   

In [3]:
import os
import numpy as np

# Paths to the combined images and labels
combined_images_path = './combined_images'
labels_path = './labels'

# Function to load combined images and labels
def load_combined_data(record_list):
    combined_images = []
    labels = []
    for record_name in record_list:
        combined_images_file = os.path.join(combined_images_path, f'{record_name}_combined_images.npy')
        labels_file = os.path.join(labels_path, f'{record_name}_labels.npy')
        
        combined_images_record = np.load(combined_images_file)
        labels_record = np.load(labels_file)
        
        combined_images.append(combined_images_record)
        labels.append(labels_record)
    
    # Concatenate all records into a single array
    combined_images = np.concatenate(combined_images, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    return combined_images, labels

# List of records to process
record_list = ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109',
               '111', '112', '113', '114', '115', '116', '117', '118', '119', '121',
               '122', '123', '124', '200', '201', '202', '203', '205', '207', '208',
               '209', '210', '212', '213', '214', '215', '217', '219', '220', '221',
               '222', '223', '228', '230', '231', '232', '233', '234']

# Load the data
images, labels = load_combined_data(record_list)

# Check the shapes of the loaded data
print(f'Combined images shape: {images.shape}')
print(f'Labels shape: {labels.shape}')


Combined images shape: (112550, 128, 128, 3)
Labels shape: (112550,)


In [3]:
import os
import numpy as np
import wfdb
from collections import Counter

# Define paths
data_path = '/home/researchgroup/mahjabeen_workspace/research/CLINet-ECG-Classification-2024/data/mit-bih/mitbih_database/' # Update this path to your data location

# List of records to process, excluding those with pacemakers
record_list = ['100', '101', '103', '105', '106', '108', '109', '111', '112', '113', 
               '114', '115', '116', '117', '118', '119', '121', '122', '123', '124', 
               '200', '201', '202', '203', '205', '207', '208', '209', '210', '212', 
               '213', '214', '215', '219', '220', '221', '222', '223', '228', '230', 
               '231', '232', '233', '234']  # Excluding 102, 104, 107, 217

# Define the AAMI class mapping
aami_mapping = {
    'N': 'Normal',
    'L': 'Normal',
    'R': 'Normal',
    'e': 'Normal',
    'j': 'Normal',
    'A': 'SVEB',
    'a': 'SVEB',
    'J': 'SVEB',
    'S': 'SVEB',
    'V': 'VEB',
    'E': 'VEB',
    'F': 'Fusion',
    '/': 'Unknown',
    'f': 'Unknown',
    'Q': 'Unknown'
}

# Function to count labels in each record
def count_labels_in_record(record_name):
    try:
        annotation = wfdb.rdann(os.path.join(data_path, record_name), 'atr')
        labels = annotation.symbol  # Annotation symbols (labels)
        aami_labels = [aami_mapping.get(label, 'Unknown') for label in labels]
        return Counter(aami_labels)
    except Exception as e:
        print(f'Error processing record {record_name}: {e}')
        return None

# Process each record and display the counts
total_labels = Counter()
for record_name in record_list:
    print(f'Processing record: {record_name}')
    label_counts = count_labels_in_record(record_name)
    if label_counts:
        print(f'Label counts for record {record_name}: {label_counts}')
        total_labels.update(label_counts)

print(f'Total label counts after exclusion: {total_labels}')


Processing record: 100
Label counts for record 100: Counter({'Normal': 2239, 'SVEB': 33, 'Unknown': 1, 'VEB': 1})
Processing record: 101
Label counts for record 101: Counter({'Normal': 1860, 'Unknown': 11, 'SVEB': 3})
Processing record: 103
Label counts for record 103: Counter({'Normal': 2082, 'Unknown': 7, 'SVEB': 2})
Processing record: 105
Label counts for record 105: Counter({'Normal': 2526, 'Unknown': 124, 'VEB': 41})
Processing record: 106
Label counts for record 106: Counter({'Normal': 1507, 'VEB': 520, 'Unknown': 71})
Processing record: 108
Label counts for record 108: Counter({'Normal': 1740, 'Unknown': 61, 'VEB': 17, 'SVEB': 4, 'Fusion': 2})
Processing record: 109
Label counts for record 109: Counter({'Normal': 2492, 'VEB': 38, 'Unknown': 3, 'Fusion': 2})
Processing record: 111
Label counts for record 111: Counter({'Normal': 2123, 'Unknown': 9, 'VEB': 1})
Processing record: 112
Label counts for record 112: Counter({'Normal': 2537, 'Unknown': 11, 'SVEB': 2})
Processing record: 

In [4]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [7]:
combined_images = np.load('./combined_images')
labels = np.load('./labels')


IsADirectoryError: [Errno 21] Is a directory: './combined_images'

In [5]:
import os
import numpy as np

# Define the directories containing the .npy files
combined_images_dir = './combined_images'
labels_dir = './labels'

# Initialize lists to store the loaded data
all_combined_images = []
all_labels = []

# Load combined images from each .npy file in the directory
for filename in os.listdir(combined_images_dir):
    if filename.endswith('.npy'):
        file_path = os.path.join(combined_images_dir, filename)
        images = np.load(file_path)
        all_combined_images.append(images)
        print(f'Loaded {filename} with shape: {images.shape}')

# Load labels from each .npy file in the directory
for filename in os.listdir(labels_dir):
    if filename.endswith('.npy'):
        file_path = os.path.join(labels_dir, filename)
        labels = np.load(file_path)
        all_labels.append(labels)
        print(f'Loaded {filename} with shape: {labels.shape}')

# Concatenate all images and labels into single arrays
combined_images = np.concatenate(all_combined_images, axis=0)
labels = np.concatenate(all_labels, axis=0)

print(f'Combined images shape: {combined_images.shape}')
print(f'Labels shape: {labels.shape}')

Loaded 217_combined_images.npy with shape: (2279, 128, 128, 3)
Loaded 122_combined_images.npy with shape: (2476, 128, 128, 3)
Loaded 107_combined_images.npy with shape: (2138, 128, 128, 3)
Loaded 105_combined_images.npy with shape: (2690, 128, 128, 3)
Loaded 106_combined_images.npy with shape: (2097, 128, 128, 3)
Loaded 101_combined_images.npy with shape: (1872, 128, 128, 3)
Loaded 212_combined_images.npy with shape: (2761, 128, 128, 3)
Loaded 220_combined_images.npy with shape: (2066, 128, 128, 3)
Loaded 116_combined_images.npy with shape: (2420, 128, 128, 3)
Loaded 208_combined_images.npy with shape: (3037, 128, 128, 3)
Loaded 108_combined_images.npy with shape: (1822, 128, 128, 3)
Loaded 200_combined_images.npy with shape: (2790, 128, 128, 3)
Loaded 210_combined_images.npy with shape: (2682, 128, 128, 3)
Loaded 223_combined_images.npy with shape: (2641, 128, 128, 3)
Loaded 115_combined_images.npy with shape: (1960, 128, 128, 3)
Loaded 234_combined_images.npy with shape: (2762, 128, 

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_images, labels, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Print shapes
print(f'Training data shape: {X_train.shape}, {y_train.shape}')
print(f'Validation data shape: {X_val.shape}, {y_val.shape}')
print(f'Testing data shape: {X_test.shape}, {y_test.shape}')

# Define paths for saving the data
save_dir = './split_data'
os.makedirs(save_dir, exist_ok=True)

# Save the training, validation, and test sets
np.save(os.path.join(save_dir, 'X_train.npy'), X_train)
np.save(os.path.join(save_dir, 'y_train.npy'), y_train)
np.save(os.path.join(save_dir, 'X_val.npy'), X_val)
np.save(os.path.join(save_dir, 'y_val.npy'), y_val)
np.save(os.path.join(save_dir, 'X_test.npy'), X_test)
np.save(os.path.join(save_dir, 'y_test.npy'), y_test)

print("Data has been saved successfully.")



: 

In [4]:
#After crash, reloading into smaller batches.
import os
import numpy as np

# Define the directories containing the .npy files
combined_images_dir = './combined_images'
labels_dir = './labels'

# Function to load data in batches
def load_data_in_batches(batch_size, combined_images_dir, labels_dir):
    all_combined_images = []
    all_labels = []
    
    # Load combined images and labels from each .npy file in batches
    for filename in os.listdir(combined_images_dir):
        if filename.endswith('.npy'):
            images_path = os.path.join(combined_images_dir, filename)
            labels_path = os.path.join(labels_dir, filename)
            
            if not os.path.exists(labels_path):
                print(f'Labels file for {filename} not found.')
                continue
            
            images = np.load(images_path)
            labels = np.load(labels_path)
            
            print(f'Loaded {filename} with image shape: {images.shape} and label shape: {labels.shape}')
            
            # Determine the number of batches
            num_batches = len(images) // batch_size + (1 if len(images) % batch_size != 0 else 0)
            
            for i in range(num_batches):
                batch_images = images[i * batch_size:(i + 1) * batch_size]
                batch_labels = labels[i * batch_size:(i + 1) * batch_size]
                
                all_combined_images.append(batch_images)
                all_labels.append(batch_labels)
                
                print(f'Processed batch {i+1}/{num_batches} for {filename}')
    
    # Concatenate all batches into single arrays
    combined_images = np.concatenate(all_combined_images, axis=0)
    labels = np.concatenate(all_labels, axis=0)
    
    return combined_images, labels

# Define batch size
batch_size = 1000

# Load data in batches
combined_images, labels = load_data_in_batches(batch_size, combined_images_dir, labels_dir)

print(f'Combined images shape: {combined_images.shape}')
print(f'Labels shape: {labels.shape}')



Labels file for 217_combined_images.npy not found.
Labels file for 122_combined_images.npy not found.
Labels file for 107_combined_images.npy not found.
Labels file for 105_combined_images.npy not found.
Labels file for 106_combined_images.npy not found.
Labels file for 101_combined_images.npy not found.
Labels file for 212_combined_images.npy not found.
Labels file for 220_combined_images.npy not found.
Labels file for 116_combined_images.npy not found.
Labels file for 208_combined_images.npy not found.
Labels file for 108_combined_images.npy not found.
Labels file for 200_combined_images.npy not found.
Labels file for 210_combined_images.npy not found.
Labels file for 223_combined_images.npy not found.
Labels file for 115_combined_images.npy not found.
Labels file for 234_combined_images.npy not found.
Labels file for 202_combined_images.npy not found.
Labels file for 233_combined_images.npy not found.
Labels file for 114_combined_images.npy not found.
Labels file for 104_combined_im

ValueError: need at least one array to concatenate

In [5]:
#debug
import os
import numpy as np

# Define the directories containing the .npy files
combined_images_dir = './combined_images'
labels_dir = './labels'

# Function to load data in batches
def load_data_in_batches(batch_size, combined_images_dir, labels_dir):
    all_combined_images = []
    all_labels = []
    missing_files = []

    # Load combined images and labels from each .npy file in batches
    for filename in os.listdir(combined_images_dir):
        if filename.endswith('.npy'):
            images_path = os.path.join(combined_images_dir, filename)
            labels_path = os.path.join(labels_dir, filename)
            
            if not os.path.exists(labels_path):
                print(f'Labels file for {filename} not found.')
                missing_files.append(filename)
                continue
            
            images = np.load(images_path)
            labels = np.load(labels_path)
            
            print(f'Loaded {filename} with image shape: {images.shape} and label shape: {labels.shape}')
            
            # Determine the number of batches
            num_batches = len(images) // batch_size + (1 if len(images) % batch_size != 0 else 0)
            
            for i in range(num_batches):
                batch_images = images[i * batch_size:(i + 1) * batch_size]
                batch_labels = labels[i * batch_size:(i + 1) * batch_size]
                
                all_combined_images.append(batch_images)
                all_labels.append(batch_labels)
                
                print(f'Processed batch {i+1}/{num_batches} for {filename}')
    
    # Check if any data was loaded
    if not all_combined_images or not all_labels:
        raise ValueError("No data was loaded. Please check the filenames and paths.")
    
    # Concatenate all batches into single arrays
    combined_images = np.concatenate(all_combined_images, axis=0)
    labels = np.concatenate(all_labels, axis=0)
    
    return combined_images, labels, missing_files

# Define batch size
batch_size = 1000

# Load data in batches
combined_images, labels, missing_files = load_data_in_batches(batch_size, combined_images_dir, labels_dir)

# Log missing files
if missing_files:
    print("Missing files:")
    for file in missing_files:
        print(file)
else:
    print("All files matched successfully.")

print(f'Combined images shape: {combined_images.shape}')
print(f'Labels shape: {labels.shape}')


Labels file for 217_combined_images.npy not found.
Labels file for 122_combined_images.npy not found.
Labels file for 107_combined_images.npy not found.
Labels file for 105_combined_images.npy not found.
Labels file for 106_combined_images.npy not found.
Labels file for 101_combined_images.npy not found.
Labels file for 212_combined_images.npy not found.
Labels file for 220_combined_images.npy not found.
Labels file for 116_combined_images.npy not found.
Labels file for 208_combined_images.npy not found.
Labels file for 108_combined_images.npy not found.
Labels file for 200_combined_images.npy not found.
Labels file for 210_combined_images.npy not found.
Labels file for 223_combined_images.npy not found.
Labels file for 115_combined_images.npy not found.
Labels file for 234_combined_images.npy not found.
Labels file for 202_combined_images.npy not found.
Labels file for 233_combined_images.npy not found.
Labels file for 114_combined_images.npy not found.
Labels file for 104_combined_im

ValueError: No data was loaded. Please check the filenames and paths.